In [1]:
from scipy.optimize import curve_fit
import datetime as dt
import pandas as pd
import numpy as np
import altair as alt

data = pd.read_csv("corona_infections.csv", header=0, names=["day", "cases"])
data["day"] = [dt.datetime.strptime(d + ' 2020', "%b %d %Y") for d in data["day"]]
data.tail(3)

,day,cases
52,2020-04-07,107663
53,2020-04-08,113296
54,2020-04-09,118235


In [2]:
infections = alt.Chart(data).mark_line(point=True).encode(
    alt.X("monthdate(day):O", title="Tag"),
    alt.Y("cases:Q", title="Corona-Fallzahlen"))
infections.properties(width=850, height=300)

alt.Chart(...)

In [3]:
first_day_with_quarantine_effect = dt.datetime(year=2020, month=3, day=28)
cases_since_quarantine = np.array(
    data[data["day"] >= first_day_with_quarantine_effect]["cases"])
day_no_since_quarantine = np.array(
    [d.toordinal() for d in data[data["day"] >= first_day_with_quarantine_effect]["day"]])
print(day_no_since_quarantine)
print(cases_since_quarantine)

[737512 737513 737514 737515 737516 737517 737518 737519 737520 737521
 737522 737523 737524]
[ 57695  62435  66885  71808  77981  84794  91159  96092 100123 103375
 107663 113296 118235]


In [4]:
def corona_curve(x, b0, x0, k, s):
    return s * 1 / (1 + np.exp(-1 * k * s * (x - x0)) * (s / b0 - 1))

print("mins:", [0, day_no_since_quarantine[0], 1e-12, cases_since_quarantine[-1]])
print("maxs:", [cases_since_quarantine[-1], dt.datetime(year=2021, month=6, day=1).toordinal(), 1e2, 8.35e7])

params, _ = curve_fit(
    corona_curve, 
    xdata=day_no_since_quarantine, 
    ydata=cases_since_quarantine, 
    p0=[cases_since_quarantine[0], dt.datetime(year=2020, month=3, day=28).toordinal(), 8e-9, 5.6e7],
    bounds=(
        [0, day_no_since_quarantine[0], 1e-11, cases_since_quarantine[-1]], 
        [cases_since_quarantine[-1], dt.datetime(year=2021, month=6, day=1).toordinal(), 1e-8, 8.35e7])
)
print(params)

mins: [0, 737512, 1e-12, 118235]
maxs: [118235, 737942, 100.0, 83500000.0]
[8.00117453e+04 7.37516456e+05 2.13452700e-09 2.61338131e+07]


In [5]:
data = data[data["day"] >= dt.datetime(year=2020, month=3, day=1)]
projection_data = data.copy()
projection_data = projection_data.drop(["cases"], axis=1)
projection_data["curve"] = [corona_curve(d.toordinal(), *params) for d in projection_data["day"]]
for i in range(10):
    projection_data = projection_data.append(pd.DataFrame(
        [[list(projection_data["day"])[-1] + dt.timedelta(days=1), 
          corona_curve((list(projection_data["day"])[-1] + dt.timedelta(days=1)).toordinal(), *params)]],
        columns=("day", "curve")
    ), ignore_index=True)
projection_data

,day,curve
0,2020-03-01,13873.591491
1,2020-03-02,14669.052689
2,2020-03-03,15510.095652
3,2020-03-04,16399.329125
4,2020-03-05,17339.510783
5,2020-03-06,18333.555685
6,2020-03-07,19384.545205
7,2020-03-08,20495.736473
8,2020-03-09,21670.572330
9,2020-03-10,22912.691849


In [6]:
infections_chart = alt.Chart(data).mark_line(point=True, color="red").encode(
    alt.X("monthdate(day):O", title="Tag"),
    alt.Y("cases:Q", title="Corona-Fallzahlen"))
projection_chart = alt.Chart(projection_data).mark_line(point=False).encode(
    alt.X("monthdate(day):O", title="Tag"),
    alt.Y("curve:Q", title="Fallzahlen (Projektion)"))
(projection_chart + infections_chart).properties(width=600, height=400)

alt.LayerChart(...)

In [7]:
bev_de = 83019213
infizierte_pro_tag = ["{:.0f}".format(bev_de*0.7/(365*dauer)) for dauer in [0.5, 1, 1.5]]
infizierte_pro_tag

['318430', '159215', '106143']

In [8]:
data = pd.read_csv("corona_infections.csv", header=0, names=["day", "cases"])
data["day"] = [dt.datetime.strptime(d + ' 2020', "%b %d %Y") for d in data["day"]]
md1 = dt.datetime(year=2020, month=3, day=20)
md2 = dt.datetime(year=2020, month=3, day=28)
c0 = np.array(data[data["day"] <= md1]["cases"])
d0 = np.array([d.toordinal() for d in data[data["day"] <= md1]["day"]])
c1 = np.array(data[(md1 <= data["day"])][(data["day"] <= md2)]["cases"])
d1 = np.array([d.toordinal() for d in data[md1 <= data["day"]][data["day"] <= md2]["day"]])
c2 = np.array(data[md2 <= data["day"]]["cases"])
d2 = np.array([d.toordinal() for d in data[md2 <= data["day"]]["day"]])
par0, _ = curve_fit(
    corona_curve, 
    xdata=d0, 
    ydata=c0, 
    p0=[c0[0], dt.datetime(year=2020, month=3, day=1).toordinal(), 8e-9, 5.6e7],
    bounds=(
        [0, d0[0], 1e-11, c0[-1]], 
        [c0[-1], dt.datetime(year=2021, month=6, day=1).toordinal(), 1e-8, 8.35e7])
)
par1, _ = curve_fit(
    corona_curve, 
    xdata=d1, 
    ydata=c1, 
    p0=[c1[0], dt.datetime(year=2020, month=3, day=20).toordinal(), 8e-9, 5.6e7],
    bounds=(
        [0, d1[0], 1e-11, c1[-1]], 
        [c1[-1], dt.datetime(year=2021, month=6, day=1).toordinal(), 1e-8, 8.35e7])
)
par2, _ = curve_fit(
    corona_curve, 
    xdata=d2, 
    ydata=c2, 
    p0=[c2[0], dt.datetime(year=2020, month=3, day=28).toordinal(), 8e-9, 5.6e7],
    bounds=(
        [0, d2[0], 1e-11, c2[-1]], 
        [c2[-1], dt.datetime(year=2021, month=6, day=1).toordinal(), 1e-8, 8.35e7])
)
print(par0, "\n", par1, "\n", par2)
data = data[data["day"] >= dt.datetime(year=2020, month=3, day=1)]
data = data.rename(columns={"day": "Tag", "cases": "Infektionszahlen"})
data["Typ"] = ["Fallzahlen" for d in data["Tag"]]
data["Typ"] = data["Typ"].astype(str)
data_count = len(data) + 15
for par, t in [(par0, "Ungebremste Ausbreitung"), 
               (par1, "Mit Ausgangsbeschränkungen"), 
               (par2, "Mit Kontaktverbot")]:
    for i in range(data_count):
        curve_value = corona_curve((list(data["Tag"])[0] + dt.timedelta(days=i)).toordinal(), *par)
        data = data.append(pd.DataFrame(
            [[list(data["Tag"])[0] + dt.timedelta(days=i), 
              curve_value if curve_value < 250000 else None,
              t]],
            columns=("Tag", "Infektionszahlen", "Typ")
        ), ignore_index=True)
data = data.tail(len(data)-data_count+15).append(data.head(data_count-15), ignore_index=True)
line = alt.Chart(data).mark_line(point=True).encode(
    alt.X("monthdate(Tag)", title="Tag"),
    alt.Y("Infektionszahlen:Q", title="Infektionszahlen"),
    color=alt.Color("Typ:N", 
                    scale=alt.Scale(scheme="set1"),
                    legend=alt.Legend(
        orient="none", legendX=20, legendY=20,
        fillColor="white", strokeColor="black", cornerRadius=7, padding=6,
        title="Prognosen nach Maßnahmen"))
)
nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['Tag'], empty='none')
selectors = alt.Chart(data).mark_point().encode(
    alt.X("monthdate(Tag)", title="Tag"),
    opacity=alt.value(0),
).add_selection(nearest)
points = line.mark_point().encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
)
text = line.mark_text(align='left', dx=5, dy=5).encode(
    text=alt.condition(nearest, 'Infektionszahlen:Q', alt.value(' '))
)
rules = alt.Chart(data).mark_rule(color='gray').encode(
    alt.X("monthdate(Tag)", title="Tag"),
).transform_filter(nearest)
alt.layer(
    line, selectors, points, rules, text
).properties(width=850, height=400)

<ipython-input-8-4b3cd215fe44>:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c1 = np.array(data[(md1 <= data["day"])][(data["day"] <= md2)]["cases"])
<ipython-input-8-4b3cd215fe44>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  d1 = np.array([d.toordinal() for d in data[md1 <= data["day"]][data["day"] <= md2]["day"]])


[1.82759729e+02 7.37485098e+05 7.94076382e-09 3.12653509e+07] 
 [2.58995320e+04 7.37506189e+05 4.71178611e-09 2.93265889e+07] 
 [8.00117453e+04 7.37516456e+05 2.13452700e-09 2.61338131e+07]


alt.LayerChart(...)